## 2.Data access

**Requirement:** First run #1

The service script stores a temporary file for its listener URI. We can find that by grepping for our server pattern.

In [1]:
import glob
server_pattern="*dkrz.de*"
cloudify_uris=[
    "http://"+':'.join(cu.split('_'))
    for cu in sorted(glob.glob(
        server_pattern
    ))
]
#use the first
cloudify_uri=cloudify_uris[0]
print(cloudify_uri)

http://l40157.lvt.dkrz.de:9000


### Available Datasets from the API

We recommend to use intake or stac for accesing data. We can however also build the dataset URIs with the API.
1. We get the *name* of the datasets via the `/datasets` API
1. We insert it to work with the `/zarr` API, e.g. in xarray:

In [18]:
import requests
datasets=eval(requests.get(cloudify_uri+"/datasets").text)
datasets

['example']

### Xarray

In [19]:
dsname=datasets[0]
zarr_url='/'.join([cloudify_uri,"datasets",dsname,"zarr"])
print(zarr_url)

http://l40157.lvt.dkrz.de:9000/datasets/example/zarr


In [4]:
import xarray as xr
ds=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options
)

In [5]:
ds

<xarray.Dataset> Size: 314MB
Dimensions:    (lat: 192, time: 1032, bnds: 2, lon: 384)
Coordinates:
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 8kB 2015-01-16T12:00:00 ... 2100-12-16T1...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 3MB dask.array<chunksize=(60, 192, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 6MB dask.array<chunksize=(60, 384, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 304MB dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] 17kB dask.array<chunksize=(1032, 2), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

### Intake

We have to tell the python programs to do not verify ssl certificates for our purposes.

**All** datasets available through the app are collected in an intake catalog.

storage_options=dict(verify_ssl=False)

In [6]:
intake_url='/'.join([cloudify_uri,"intake.yaml"])
print(intake_url)

http://l40157.lvt.dkrz.de:9000/intake.yaml


In [7]:
import intake
cat=intake.open_catalog(
    intake_url,
    storage_options=storage_options
)
list(cat)

['example']

In [8]:
cat[dsname]

example:
  args:
    consolidated: true
    urlpath: https://l40157.lvt.dkrz.de:9000/datasets/example/kerchunk
  description: ''
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: http://l40157.lvt.dkrz.de:9000


The intake plugin provides `urlpath` with *https* instead of *http*. So on Levante, we have to modify that:

In [9]:
cat[dsname](
#    method="zarr",
    storage_options=storage_options,
    urlpath=cat[dsname].urlpath.replace('https','http').replace('kerchunk','zarr')
).to_dask()

/work/bm0021/conda-envs/cloudify/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


<xarray.Dataset> Size: 314MB
Dimensions:    (lat: 192, time: 1032, bnds: 2, lon: 384)
Coordinates:
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 8kB 2015-01-16T12:00:00 ... 2100-12-16T1...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 3MB dask.array<chunksize=(60, 192, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 6MB dask.array<chunksize=(60, 384, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 304MB dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] 17kB dask.array<chunksize=(1032, 2), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

### Stac

For each dataset, a stac item is generated with enriched metadata. The URL for this API is similar to the *zarr*-URL:

In [10]:
stac_url=zarr_url.replace('/zarr','/stac')

In [11]:
import pystac
stacitem=pystac.item.Item.from_file(stac_url)
stacitem

<Item id=example>

In [12]:
stacitem.assets

{'data': <Asset href=https://eerie.cloud.dkrz.de/datasets/example/zarr>,
 'xarray_view': <Asset href=https://eerie.cloud.dkrz.de/datasets/example/>}

The stac API is right now hard-coded for 'eerie.cloud'. In theory, we could get to the data with xarray and the *href* asset.

### cdo

We are developing a cdo version that can read cloudified data with netcdf-zarr via http. The built netcdf is a testversion.

- add your self signed ssl certificate to the following bash code block
- add the zarr_url as it has to be hard coded
- check if the HDF5_PLUGIN_PATH exists

In [13]:
print(zarr_url)

http://l40157.lvt.dkrz.de:9000/datasets/example/zarr


In [14]:
%%bash
#export SSL_CERT_FILE=/work/bm0021/k204210/cloudify/workshop/cert.pem 
cdo="/work/bm0021/cdo_incl_cmor/cdo-test_cmortest_gcc/bin/cdo"
zarr_prefix="#mode=zarr,zoh"
infile="http://l40157.lvt.dkrz.de:9000/datasets/example/zarr${zarr_prefix}"
#
$cdo sinfo $infile

cdi  warning (cdf_read_coordinates): Time varying grid x-bounds unsupported, skipped!
cdi  warning (cdf_read_coordinates): Time varying grid y-bounds unsupported, skipped!


   File format : NCZarr filter
    -1 : Institut Source   T Steptype Levels Num    Points Num Dtype : Parameter ID
     1 : unknown  MPI-ESM1.2-HR v instant       1   1     73728   1  F32f : -1            
   Grid coordinates :
     1 : gaussian                 : points=73728 (384x192)  F96
                              lon : 0 to 359.0625 by 0.9375 degrees_east  circular
                              lat : -89.28423 to 89.28423 degrees_north
   Vertical coordinates :
     1 : surface                  : levels=1
   Time coordinate :
                             time : 1032 steps
     RefTime =  1850-01-01 00:00:00  Units = days  Calendar = proleptic_gregorian  Bounds = true
  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss
  2015-01-16 12:00:00  2015-02-15 00:00:00  2015-03-16 12:00:00  2015-04-16 00:00:00
  2015-05-16 12:00:00  2015-06-16 00:00:00  2015-07-16 12:00:00  2015-08-16 12:00:00
  2015-09-16 00:00:00  2015-10-16 12:00:00  2015-11-16 00:00:0

When working with cdo on cloudified data, we have to select data with the **select** operator:

In [15]:
%%bash
#export SSL_CERT_FILE=/work/bm0021/k204210/cloudify/workshop/cert.pem 
cdo="/work/bm0021/cdo_incl_cmor/cdo-test_cmortest_gcc/bin/cdo"
zarr_prefix="#mode=zarr,zoh"
infile="http://l40157.lvt.dkrz.de:9000/datasets/example/zarr${zarr_prefix}"
#
$cdo info -select,name=tas,timestep=1 $infile

cdi  warning (cdf_read_coordinates): Time varying grid x-bounds unsupported, skipped!
cdi  warning (cdf_read_coordinates): Time varying grid y-bounds unsupported, skipped!


    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 2015-01-16 12:00:00       0    73728       0 :      231.20      277.55      309.37 : -1            
